In [1]:
# import relevant libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import torch
print(torch.__version__)
from torch import nn
from torch import optim 
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from torch.utils.tensorboard import SummaryWriter # TensorBoard to plot losses

from dataset_creator import EITDataset
from network_models import EITNet

# configure device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # use GPU if available
print(f"Using {device} device")


2.6.0+cu124
Using cpu device


In [2]:
import os

# Set the working directory to the script's directory
script_dir = os.path.dirname('/home/asimov/Soft-Tactile-Sensing-For-Robotic-Manipulation/')
os.chdir(script_dir)
print("Changed working directory to:", os.getcwd())


Changed working directory to: /home/asimov/Soft-Tactile-Sensing-For-Robotic-Manipulation


In [3]:
## Configure dataset and dataloader
data_folder = "Readings/Funky/"

# find data_folder
if not os.path.exists(data_folder):
    raise Exception(f"Data folder {data_folder} does not exist")

# Create dataset instance
dataset = EITDataset(data_folder)

# Split dataset into training and validation sets
generator1 = torch.Generator().manual_seed(91) # rng seed for splitting into train and val
split = random_split(dataset, [0.8, 0.2], generator=generator1) # split dataset into 80% train and 20% val

train_dataset = split[0] # get training dataset
val_dataset = split[1] # get validation dataset

print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

train_dataloader = DataLoader(train_dataset, batch_size = 16, shuffle = True) # create dataloader for training dataset
val_dataloader = DataLoader(val_dataset, batch_size = 16, shuffle = True) # create dataloader for validation dataset

Training dataset size: 608
Validation dataset size: 151


In [4]:
# look at 1 sample
sample = dataset[1]
readings = sample[0]


In [5]:
## Initialise network
# Define network
model = EITNet().to(device) # move model to device
print(model)

EITNet(
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (shape_head): Linear(in_features=64, out_features=5, bias=True)
  (pos_head): Linear(in_features=64, out_features=3, bias=True)
  (orient_head): Linear(in_features=64, out_features=2, bias=True)
)


In [6]:
# define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)


In [7]:
## Training 1 epoch

def train_epoch(epoch_index, tb_writer):
    running_loss = 0.0
    last_epoch_loss = 0.0
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(train_dataloader):
        readings, labels = data
        readings = readings.to(device)
        labels = labels.to(device)
        
        shape_labels = labels[:, 0]
        position_labels = labels[:, 1]
        orientation_labels = labels[:, 2]
        
        # Zero your gradients for every batch
        optimiser.zero_grad()
        
        # Forward pass through the network
        shape_logits, position_logits, orientation_logits = model(readings)
        
        
        # Compute the total loss and its gradients
        # loss for each head
        loss_shape = criterion(shape_logits, shape_labels)
        loss_position = criterion(position_logits, position_labels)
        loss_orientation = criterion(orientation_logits, orientation_labels)
        
        # total loss
        loss = loss_shape + loss_position + loss_orientation
        
        loss.backward() # grads

        # Adjust learning weights
        optimiser.step()
        
        # Gather data and report to TensorBoard every 10 batches
        running_loss += loss.item()
        if i % 10 == 9:
            last_loss = running_loss / 10 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_dataloader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
    return last_loss



In [8]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S') # timestamp for TensorBoard logs
writer = SummaryWriter('4M25_modelling/runs/model_test_{}'.format(timestamp)) # create TensorBoard writer for each run
epoch_number = 0 # initialise epoch number

In [9]:
#training loop
EPOCHS = 50 # number of epochs

best_vloss = 1_000_000. # initialise best validation loss to a large value

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))
    
    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_epoch(epoch_number, writer)
    
    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(val_dataloader):
        vreadings, vlabels = vdata
        vreadings = vreadings.to(device)
        vlabels = vlabels.to(device)
        vshape_labels = vlabels[:, 0]
        vposition_labels = vlabels[:, 1]
        vorientation_labels = vlabels[:, 2]
        
        vshape_logits, vposition_logits, vorientation_logits = model(vreadings)
        
        vloss_shape = criterion(vshape_logits, vshape_labels)
        vloss_position = criterion(vposition_logits, vposition_labels)
        vloss_orientation = criterion(vorientation_logits, vorientation_labels)
        
        # total loss
        vloss = vloss_shape + vloss_position + vloss_orientation
        
        running_vloss += vloss
    
    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
    
    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()
    
    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = '4M25_modelling/trained_models/model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)
    
    epoch_number += 1

EPOCH 1:
  batch 10 loss: 3.2618546009063722
  batch 20 loss: 2.7697469711303713
  batch 30 loss: 2.477886438369751
LOSS train 2.477886438369751 valid 1.8569942712783813
EPOCH 2:
  batch 10 loss: 1.9621262788772582
  batch 20 loss: 1.812419593334198
  batch 30 loss: 1.8162100315093994
LOSS train 1.8162100315093994 valid 1.4684627056121826
EPOCH 3:
  batch 10 loss: 1.611629021167755
  batch 20 loss: 1.4031672835350038
  batch 30 loss: 1.37778902053833
LOSS train 1.37778902053833 valid 1.2033379077911377
EPOCH 4:
  batch 10 loss: 1.2318739652633668
  batch 20 loss: 1.202217435836792
  batch 30 loss: 1.1935820817947387
LOSS train 1.1935820817947387 valid 0.9958569407463074
EPOCH 5:
  batch 10 loss: 0.9864594399929046
  batch 20 loss: 0.9858055531978607
  batch 30 loss: 0.8946448504924774
LOSS train 0.8946448504924774 valid 0.9048153162002563
EPOCH 6:
  batch 10 loss: 0.8120444774627685
  batch 20 loss: 0.7788848280906677
  batch 30 loss: 0.8067529797554016
LOSS train 0.8067529797554016 va

In [11]:
## Testing
# Load the best model
model.load_state_dict(torch.load(model_path))
print('Loaded model from', model_path)
model.eval()

# Test the model on the validation set
correct = 0
total = 0

with torch.no_grad():
    for data in val_dataloader:
        readings, labels = data
        readings = readings.to(device)
        labels = labels.to(device)
        shape_labels = labels[:, 0]
        position_labels = labels[:, 1]
        orientation_labels = labels[:, 2]
        
        shape_logits, position_logits, orientation_logits = model(readings)
        
        _, shape_pred = torch.max(shape_logits, 1)
        _, position_pred = torch.max(position_logits, 1)
        _, orientation_pred = torch.max(orientation_logits, 1)
        
        total += labels.size(0)
        correct += ((shape_pred == shape_labels) & (position_pred == position_labels) & (orientation_pred == orientation_labels)).sum().item()

print('Accuracy of the network on the validation set: %d %%' % (100 * correct / total))


Loaded model from 4M25_modelling/trained_models/model_20250317_162245_13
Accuracy of the network on the validation set: 78 %
